# IBM Data Science Specialization Course 9

This notebook will be mainly used for the capstone project.

In [125]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Course 9 week 3 assignment: Toronto

Scrape the Wikipedia table at https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M (retrieved 24 Feb 2019) into a pandas DataFrame.

## First try: parse Wikipedia using py-wikimarkup

Initial how-to instructions from https://stackoverflow.com/questions/15724034/how-to-convert-wikipedia-wikitable-to-python-pandas-dataframe (retrieved 24 Feb 2019).

In [126]:
!pip install py-wikimarkup
!pip install pyquery

Requirement not upgraded as not directly required: py-wikimarkup in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: pyquery in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: cssselect>0.7.9 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyquery)
Requirement not upgraded as not directly required: lxml>=2.1 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from pyquery)


In [127]:
import wikimarkup
import pandas as pd
from pyquery import PyQuery

def get_tables(wiki):
    html = PyQuery(wikimarkup.parse(wiki))
    frames = []
    for table in html('table'):
        data = [[x.text.strip() for x in row]
                for row in table.getchildren()]
        df = pd.DataFrame(data[1:], columns=data[0])
        frames.append(df)
    return frames


ImportError: cannot import name 'parse'

Unfortunately, a pre-installed "parse" package in IBM Watson Studio conflicts with the "parse.py" file name inside wikimarkup.

## Second try: parse Wikipedia using BeautifulSoup

Following https://medium.com/analytics-vidhya/web-scraping-wiki-tables-using-beautifulsoup-and-python-6b9ea26d8722 (retrieved 24 Feb 2019)

Gettin HTML of the Wikipedia page:

In [128]:
import requests
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
website_url = requests.get(url).text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')
print("{}\n\n   [...]\n\n{}".format(soup.prettify()[:500], soup.prettify()[-500:]))

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_po

   [...]

":"Contributors to Wikimedia projects"},"publisher":{"@type":"Organization","name":"Wikimedia Foundation, Inc.","logo":{"@type":"ImageObject","url":"https:\/\/www.wikimedia.org\/static\/images\/wmf-hor-googpub.png"}},"datePublished":"2004-03-20T10:02:13Z","dateModified":"2019-01-04T18:32:45Z","headline":"Wikimedia list article"}
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgBackendResponseTime":84,"wgHostname":"mw1265"});});
  </script>
 </bo

This library seems to function OK.

The needed data is contained in the first (and only) table, and is rendered with HTML style "wikitable sortable". Parse it:

In [129]:
My_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(My_table)[:500], str(My_table)[-500:]))

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td>

   [...]

/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/The_Queensway" title="The Queensway">The Queensway West</a>
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td>Royal York South West
</td></tr>
<tr>
<td>M8Z</td>
<td><a href="/wiki/Etobicoke" title="Etobicoke">Etobicoke</a></td>
<td><a href="/wiki/Bloor" title="Bloor">South of Bloor</a>
</td></tr>
<tr>
<td>M9Z</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
</tb

## Clean data for postal code, borough, and neighborhood

Now loop through all rows from that table. For each row:
1. Ignore if it is the header row (no "TD" tags),
2. strip leading and trailing whitespace (including line breaks),
3. Ignore if the borough is "Not assigned",
4. If the borough is given but neighborhoos is "Not assigned", fill in the borough as neighborhood name.
Finally, build a new pandas DataFrame from the list items collected.

In [130]:
l = []
unassigned_pc = []

table_rows = My_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("ignoring empty row")
    elif row[1] == 'Not assigned':
        unassigned_pc.append(row[0])
    else:
        if row[2] == 'Not assigned':
            row[2] = row[1]
            print('assigning neighborhood same as borough: {}'.format(row[2]))
        l.append(row)

print("ignoring unassigned postal codes:{}".format(unassigned_pc))
        
My_df = pd.DataFrame(l, columns=["PostalCode", "Borough", "Neighborhood"])
print("My_df.head()=\n{}".format(My_df.head()))
print("total: My_df.shape={}".format(My_df.shape))

ignoring empty row
assigning neighborhood same as borough: Queen's Park
ignoring unassigned postal codes:['M1A', 'M2A', 'M8A', 'M2B', 'M7B', 'M8B', 'M2C', 'M7C', 'M8C', 'M2E', 'M3E', 'M7E', 'M8E', 'M9E', 'M2G', 'M3G', 'M7G', 'M8G', 'M9G', 'M7H', 'M8H', 'M9H', 'M7J', 'M8J', 'M9J', 'M7K', 'M8K', 'M9K', 'M7L', 'M8L', 'M7M', 'M8M', 'M7N', 'M8N', 'M3P', 'M7P', 'M8P', 'M3R', 'M8R', 'M2S', 'M3S', 'M7S', 'M8S', 'M9S', 'M2T', 'M3T', 'M6T', 'M7T', 'M8T', 'M9T', 'M2V', 'M3V', 'M6V', 'M7V', 'M2W', 'M3W', 'M6W', 'M7W', 'M2X', 'M3X', 'M6X', 'M7X', 'M9X', 'M1Y', 'M2Y', 'M3Y', 'M5Y', 'M6Y', 'M9Y', 'M1Z', 'M2Z', 'M3Z', 'M4Z', 'M5Z', 'M6Z', 'M7Z', 'M9Z']
My_df.head()=
  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
total: My_df.shape=(212, 3)


## Aggregate neighborhoods by borough and postal code

Concatenate neighborhoods grouped by postal code and borough.

In [131]:
df_grouped = My_df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index()
print("df_grouped.head()=\n{}".format(df_grouped.head()))
print("total: df_grouped.shape={}".format(df_grouped.shape))

df_grouped.head()=
  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Rouge, Malvern
1        M1C  Scarborough  Highland Creek, Rouge Hill, Port Union
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae
total: df_grouped.shape=(103, 3)


## Result: Number of rows in Toronto neighborhood DataFrame

In [132]:
print("Number of records in cleaned, aggregated Toronto neighborhoods scraped from Wikipedia:\n{}".format(df_grouped.shape[0]))

Number of records in cleaned, aggregated Toronto neighborhoods scraped from Wikipedia:
103


## Geolocation of Toronto neighborhoods

Install geocoder library and try to get location for the neighborhoods

In [133]:
!pip install geocoder

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: i

In [134]:
import geocoder # import geocoder

# testing geocoder
lat_lng_coords = None
cnt = 0

# loop until you get the coordinates
while(lat_lng_coords is None and cnt < 10):
    print('attempt #{}...'.format(cnt))
    cnt += 1
    g = geocoder.google('Mountain View, CA')
    lat_lng_coords = g.latlng

if lat_lng_coords is None:
    print('All API calls unsuccessful')
else:
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print("Success, lat/long = {}/{}".format(latitude, longitude))


attempt #0...
attempt #1...
attempt #2...
attempt #3...
attempt #4...
attempt #5...
attempt #6...
attempt #7...
attempt #8...
attempt #9...
All API calls unsuccessful


Google doesn't seem to work well, try OpenStreetMap

In [135]:
osm_result = None
cnt = 0

# loop until you get the coordinates
while(osm_result is None and cnt < 10):
    print('attempt #{}...'.format(cnt))
    cnt += 1
    g = geocoder.osm('M1B, Toronto, Ontario, Canada')
    osm_result = g.json

if osm_result is None:
    print('All API calls unsuccessful')
else:
    print("Success:\n{}".format(g.json))

attempt #0...
Success:
{'osm_id': '18063533', 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.png', 'ok': True, 'confidence': 1, 'region': 'Ontario', 'address': 'Toronto, Ontario, M6K 1X9, Canada', 'type': 'city', 'bbox': {'southwest': [43.493963, -79.547207], 'northeast': [43.813963, -79.227207]}, 'place_rank': '16', 'country': 'Canada', 'postal': 'M6K 1X9', 'status': 'OK', 'place_id': '110816', 'quality': 'city', 'osm_type': 'node', 'city': 'Toronto', 'state': 'Ontario', 'importance': 1.066077038952242, 'accuracy': 1.066077038952242, 'raw': {'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_place_city.p.20.png', 'osm_id': '18063533', 'place_rank': '16', 'boundingbox': ['43.493963', '43.813963', '-79.547207', '-79.227207'], 'place_id': '110816', 'osm_type': 'node', 'address': {'city': 'Toronto', 'state': 'Ontario', 'country_code': 'ca', 'postcode': 'M6K 1X9', 'country': 'Canada'}, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. 

This seems to work on first try.

Write a function and see how it performs on all postal codes:

In [136]:
def get_osm_loc(param='', base_address='Toronto, Ontario, {}, Canada'):
    g = geocoder.osm(base_address.format(param))
    osm_result = g.json
    if osm_result is None:
        # API call unsuccessful
        return 0, 0
    else:
        # success, extract lat/long
        latitude = osm_result['lat']
        longitude = osm_result['lng']
        return (latitude, longitude)

print('get_osm_loc("", "Paris, France")={}'.format(get_osm_loc("", "Paris, France")))

for test_pc in ['M5G', 'M1B', 'M2H', 'M4B']:
    print('get_osm_loc("{}")={}'.format(test_pc, get_osm_loc(test_pc)))


get_osm_loc("", "Paris, France")=(48.8566101, 2.3514992)
get_osm_loc("M5G")=(0, 0)
get_osm_loc("M1B")=(43.653963, -79.387207)
get_osm_loc("M2H")=(0, 0)
get_osm_loc("M4B")=(0, 0)


Ok, OpenStreenMap resolves some but not all locations (including postal codes in Toronto).

## Try geocoder with CSV lookup as fallback

Loop through all records obtained earlier and try to get geolocation dynamically for the Toronto boroughs. After 10 tries, fall back to using a provided CSV file as lookup.

In [137]:
geo_csv_url = 'https://cocl.us/Geospatial_data'
geo_df = pd.read_csv(geo_csv_url)
print('geo_df.head()=\n{}'.format(geo_df.head()))


geo_df.head()=
  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [138]:
def get_pc_loc(pc):
    (latitude, longitude) = get_osm_loc(pc)
    if latitude == 0 and longitude == 0:
        loc_info = geo_df[geo_df['Postal Code'] == pc]
        if loc_info.shape[0] > 0:
            latitude = loc_info['Latitude'].values[0]
            longitude = loc_info['Longitude'].values[0]
            print("{}: got lat/long from CSV".format(pc))
        else:
            latitude = 0
            longitude = 0
            print("{}: could not find lat/long".format(pc))
    else:
        print("{}: got lat/long from osm".format(pc))
    return (latitude, longitude)

for test_pc in ['M5G', 'M1B', 'M2H', 'M4B']:
    print('get_pc_loc("{}")={}'.format(test_pc, get_pc_loc(test_pc)))


M5G: got lat/long from CSV
get_pc_loc("M5G")=(43.657952399999999, -79.387382599999995)
M1B: got lat/long from osm
get_pc_loc("M1B")=(43.653963, -79.387207)
M2H: got lat/long from CSV
get_pc_loc("M2H")=(43.803762200000001, -79.363451699999999)
M4B: got lat/long from CSV
get_pc_loc("M4B")=(43.706397199999998, -79.309937000000005)


This looks good, ready to put it all together.

For every row in the DataFrame that is grouped by Postal Code and Borough, apply the get_pc_local() function and put the lat/long values into a
new temporary `TempLatLong` column.

In [139]:
print("Obtaining latitudes...")
df_grouped['TempLatLong'] = df_grouped.apply(lambda x : get_pc_loc(x['PostalCode']), axis=1)


Obtaining latitudes...
M1B: got lat/long from osm
M1C: got lat/long from osm
M1E: got lat/long from CSV
M1G: got lat/long from osm
M1H: got lat/long from CSV
M1J: got lat/long from CSV
M1K: got lat/long from CSV
M1L: got lat/long from CSV
M1M: got lat/long from CSV
M1N: got lat/long from CSV
M1P: got lat/long from CSV
M1R: got lat/long from CSV
M1S: got lat/long from CSV
M1T: got lat/long from CSV
M1V: got lat/long from CSV
M1W: got lat/long from osm
M1X: got lat/long from CSV
M2H: got lat/long from CSV
M2J: got lat/long from osm
M2K: got lat/long from CSV
M2L: got lat/long from CSV
M2M: got lat/long from CSV
M2N: got lat/long from osm
M2P: got lat/long from CSV
M2R: got lat/long from CSV
M3A: got lat/long from CSV
M3B: got lat/long from CSV
M3C: got lat/long from osm
M3H: got lat/long from osm
M3J: got lat/long from osm
M3K: got lat/long from CSV
M3L: got lat/long from CSV
M3M: got lat/long from CSV
M3N: got lat/long from CSV
M4A: got lat/long from CSV
M4B: got lat/long from CSV
M4C: 

In [140]:
print('The first 10 records of the new DataFrame:')
df_grouped.head(10)

The first 10 records of the new DataFrame:


,PostalCode,Borough,Neighborhood,TempLatLong
0,M1B,Scarborough,"Rouge, Malvern","(43.653963, -79.387207)"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union","(43.653963, -79.387207)"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill","(43.7635726, -79.1887115)"
3,M1G,Scarborough,Woburn,"(43.7394839, -79.369314)"
4,M1H,Scarborough,Cedarbrae,"(43.773136, -79.2394761)"
5,M1J,Scarborough,Scarborough Village,"(43.7447342, -79.2394761)"
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park","(43.7279292, -79.2620294)"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge","(43.7111117, -79.2845772)"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West","(43.716316, -79.2394761)"
9,M1N,Scarborough,"Birch Cliff, Cliffside West","(43.692657, -79.2648481)"


Extract the lat/long coordinates from the `TempLatLong` column, and create the DataFrame in the format required by the assignment.

In [141]:
df_grouped['Latitude'] = df_grouped.apply(lambda x : x['TempLatLong'][0], axis=1)
df_grouped['Longitude'] = df_grouped.apply(lambda x : x['TempLatLong'][1], axis=1)
df_grouped = df_grouped.drop(['TempLatLong'], axis=1)

In [142]:
print('The first 10 records of the new DataFrame:')
df_grouped.head(10)

The first 10 records of the new DataFrame:


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.653963,-79.387207
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.653963,-79.387207
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.739484,-79.369314
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Explore neighborhoods

Limit to boroughs that contain the word "Toronto", then perform the same analysis as we did on the New York City data set.

In [143]:
print('All boroughs: {}'.format(df_grouped['Borough'].unique()))
df_toronto = df_grouped[df_grouped['Borough'].str.contains('Toronto')]
print('Boroughs with "Toronto" in the name: {}'.format(df_toronto['Borough'].unique()))

All boroughs: ['Scarborough' 'North York' 'East York' 'East Toronto' 'Central Toronto'
 'Downtown Toronto' 'York' 'West Toronto' "Queen's Park" 'Mississauga'
 'Etobicoke']
Boroughs with "Toronto" in the name: ['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


## Load libraries required for map preparation and rendering (folium)

In [144]:
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Libraries imported.')


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


In [146]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.68,-79.37], zoom_start=13)

neighborhoods = pd.DataFrame(df_toronto)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


# Image snapshot:

https://github.com/koeplinger/sandbox-ibm-ds/blob/master/c9w3_assg_pic1.png

In [148]:
# Copy credentials here
# CLIENT_ID = ''
# CLIENT_SECRET = ''
VERSION = '20180605' # Foursquare API version


print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


(this cell holds the authentication details)

<!--

CLIENT_ID = 'NBLOR5JJCSM43LTXYWBQYVJ5U3LMNZ2ULCHERZAZVLJTHBYA' # your Foursquare ID
CLIENT_SECRET = 'D1G4RELNK2MGSOZSO1C4DTGPYBAWHIW0MQJTXWSBTGH2JL41' # your Foursquare Secret

-->

# Get nearby venues

Using the Foursquare API, get a list of venues within 500 meters of the neighborhood lat/long center.

In [149]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


# Collect all nearby venues

In [150]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )



print(toronto_venues.shape)
toronto_venues.head()


The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


# Check distribution of venues across neighborhoods

In [151]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,71,71,71,71,71,71
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",99,99,99,99,99,99
Christie,15,15,15,15,15,15


# Remove neighborhoods with very few venues

When clustering neighborhoods by similarity of venues, it is not feasible to process neighborhoods that simply don't have many venues. In a way, this is our first cluster group: Neighborhoods with 10 or fewer venues.

In [152]:
df_toronto_hasfewvenues = (toronto_venues.groupby('Neighborhood').count() <= 10).reset_index()
df_toronto_hasfewvenues = df_toronto_hasfewvenues[df_toronto_hasfewvenues['Venue'] == True]
df_toronto_hasfewvenues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,Davisville North,True,True,True,True,True,True
17,"Forest Hill North, Forest Hill West",True,True,True,True,True,True
22,Lawrence Park,True,True,True,True,True,True
24,"Moore Park, Summerhill East",True,True,True,True,True,True
27,Rosedale,True,True,True,True,True,True
28,Roselawn,True,True,True,True,True,True
35,The Beaches,True,True,True,True,True,True


In [153]:
neighborhoods_with_few_venues = df_toronto_hasfewvenues['Neighborhood']
neighborhoods_with_few_venues

12                       Davisville North
17    Forest Hill North, Forest Hill West
22                          Lawrence Park
24            Moore Park, Summerhill East
27                               Rosedale
28                               Roselawn
35                            The Beaches
Name: Neighborhood, dtype: object

In [154]:
toronto_many_venues = toronto_venues[~toronto_venues['Neighborhood'].isin(neighborhoods_with_few_venues)]
toronto_many_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,71,71,71,71,71,71
"Brockton, Exhibition Place, Parkdale Village",100,100,100,100,100,100
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",100,100,100,100,100,100
"Cabbagetown, St. James Town",50,50,50,50,50,50
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",99,99,99,99,99,99
Christie,15,15,15,15,15,15


# Check how many categories there are

In [155]:
print('There are {} venues with {} uniques categories.'.format(len(toronto_many_venues), len(toronto_many_venues['Venue Category'].unique())))
toronto_many_venues['Venue Category'].unique()


There are 1882 venues with 229 uniques categories.


array(['Greek Restaurant', 'Ice Cream Shop', 'Cosmetics Shop',
       'Italian Restaurant', 'Yoga Studio', 'Brewery', 'Pub',
       'Fruit & Vegetable Store', 'Pizza Place', 'Restaurant', 'Juice Bar',
       'Trail', 'Bookstore', 'Diner', 'Dessert Shop', 'Bubble Tea Shop',
       'Indian Restaurant', 'Spa', 'Grocery Store', 'Bakery',
       'Caribbean Restaurant', 'Coffee Shop', 'American Restaurant',
       'Lounge', 'Liquor Store', 'Furniture / Home Store', 'Burger Joint',
       'Gym', 'Fish & Chips Shop', 'Park', 'Sushi Restaurant',
       'Steakhouse', 'Burrito Place', 'Pet Store', 'Fast Food Restaurant',
       'Movie Theater', 'Sandwich Place', 'Light Rail Station',
       'Food & Drink Shop', 'Fish Market', 'Cheese Shop', 'Café',
       'Comfort Food Restaurant', 'Neighborhood', 'Chinese Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store',
       'New American Restaurant', 'Seafood Restaurant', 'Coworking Space',
       'Music Store', 'Bar', 'Gastropub', 'Conven

# Clean up similar venues

Neighborhoods will be considered similar if they have - say - many restaurants or bars, irrespective of what restaurant or bar type they are. Simplifying the categories.

In [156]:
toronto_venues_simple = pd.DataFrame(toronto_many_venues)

def simplify_cat(df, cat_string, cat_string_simple=None):
    if cat_string_simple is None:
        cat_string_simple = cat_string
    df.loc[df['Venue Category'].str.contains(cat_string), 'Venue Category'] = cat_string_simple

simplify_cat(toronto_venues_simple, 'Restaurant')
simplify_cat(toronto_venues_simple, 'Bar')
simplify_cat(toronto_venues_simple, 'Lounge', 'Bar')
simplify_cat(toronto_venues_simple, 'Pub')
simplify_cat(toronto_venues_simple, 'Gastropub', 'Pub')
simplify_cat(toronto_venues_simple, 'Nightlife')
simplify_cat(toronto_venues_simple, 'Nightclub', 'Nightlife')
simplify_cat(toronto_venues_simple, 'Vegetable', 'Grocery Store')
simplify_cat(toronto_venues_simple, 'Deli', 'Grocery Store')
simplify_cat(toronto_venues_simple, 'Supermarket', 'Grocery Store')
simplify_cat(toronto_venues_simple, 'Joint', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Fish & Chips', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Ice Cream', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Pizza', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Dessert', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Steak', 'Restaurant')
simplify_cat(toronto_venues_simple, 'Sandwich', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Donut', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Noodle', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Salad', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Soup', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Cupcake', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Food', 'Simple Food')
simplify_cat(toronto_venues_simple, 'Tea')
simplify_cat(toronto_venues_simple, 'Food & Drink', 'Grocery Store')
simplify_cat(toronto_venues_simple, 'Coffee', 'Coffee Shop')
simplify_cat(toronto_venues_simple, 'Café', 'Coffee Shop')
simplify_cat(toronto_venues_simple, 'Convenience Store', 'Grocery Store')
simplify_cat(toronto_venues_simple, 'Lingerie', 'Adult')
simplify_cat(toronto_venues_simple, 'Strip', 'Adult')
simplify_cat(toronto_venues_simple, 'Adult', 'Adult')
simplify_cat(toronto_venues_simple, 'Art')
simplify_cat(toronto_venues_simple, 'Book')
simplify_cat(toronto_venues_simple, 'Auto')
simplify_cat(toronto_venues_simple, 'Gym')
simplify_cat(toronto_venues_simple, 'Music')
simplify_cat(toronto_venues_simple, 'Rec Center', 'Gym')
simplify_cat(toronto_venues_simple, 'Station')
simplify_cat(toronto_venues_simple, 'Stop', 'Station')

print('There now are {} venues with {} unique simplified categories.'.format(len(toronto_venues_simple), len(toronto_venues_simple['Venue Category'].unique())))
toronto_venues_simple['Venue Category'].unique()


There now are 1882 venues with 123 unique simplified categories.


array(['Restaurant', 'Simple Food', 'Cosmetics Shop', 'Yoga Studio',
       'Brewery', 'Pub', 'Grocery Store', 'Bar', 'Trail', 'Book', 'Diner',
       'Tea', 'Spa', 'Bakery', 'Coffee Shop', 'Liquor Store',
       'Furniture / Home Store', 'Gym', 'Park', 'Burrito Place',
       'Pet Store', 'Movie Theater', 'Station', 'Fish Market',
       'Cheese Shop', 'Neighborhood', 'Coworking Space', 'Music', 'Bank',
       'Clothing Store', 'Sporting Goods Shop', 'Toy / Game Store',
       'Health & Beauty Service', 'Rental Car Location', 'Gourmet Shop',
       'Farmers Market', 'Pharmacy', 'Costume Shop', 'Indoor Play Area',
       'Bagel Shop', 'Butcher', 'Jewelry Store', 'General Entertainment',
       'Gift Shop', 'Market', 'Library', 'Playground', 'Beer Store',
       'Breakfast Spot', 'Flower Shop', 'Bike Rental / Bike Share',
       'Dance Studio', 'Hobby Shop', 'Adult', 'Creperie', "Men's Store",
       'Smoke Shop', 'Dog Run', 'Art', 'Nightlife', 'Video Game Store',
       'Theater', 'Scu

# Perform one-hot encoding

With each type of venue appearing more than 10 times on average we may have enough data to compare neighborhoods. Perform one-hot encoding by venue category.

In [157]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_simple[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_simple['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()


,Yoga Studio,Adult,Antique Shop,Art,Auto,Baby Store,Bagel Shop,Bakery,Bank,Bar,...,Tanning Salon,Tea,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,University,Video Game Store,Wine Shop,Women's Store
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Confirm that the expected number of categories (from above) now shows in the columns:

In [158]:
toronto_onehot.shape

(1882, 123)

Group records by neighborhood, and build the average venue category each.

In [159]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult,Antique Shop,Art,Auto,Baby Store,Bagel Shop,Bakery,Bank,...,Tanning Salon,Tea,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,University,Video Game Store,Wine Shop,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01
1,Berczy Park,0.000000,0.000000,0.000000,0.084507,0.000000,0.000000,0.000000,0.014085,0.000000,...,0.00,0.028169,0.000000,0.000000,0.014085,0.000000,0.014085,0.000000,0.00,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.030000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.00,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.01
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.020000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.00
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.020000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
6,Central Bay Street,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.000000,0.012195,0.000000,...,0.00,0.036585,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.020202,0.000000,0.000000,0.010101,0.040404,0.000000,...,0.00,0.020202,0.000000,0.010101,0.000000,0.000000,0.000000,0.000000,0.00,0.00
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00
9,Church and Wellesley,0.011628,0.023256,0.000000,0.011628,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.034884,0.011628,0.000000,0.000000,0.000000,0.000000,0.011628,0.00,0.00


Confirm the expected number of neighborhoods (rows) and venue categories (columns):

In [160]:
toronto_grouped.shape

(31, 123)

# Inspect top-N venues

For all neighborhoods, make a visual inspection of the top 5 venue categories.

In [161]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Adelaide, King, Richmond----
         venue  freq
0   Restaurant  0.31
1  Coffee Shop  0.11
2  Simple Food  0.08
3          Bar  0.07
4          Gym  0.04


----Berczy Park----
            venue  freq
0      Restaurant  0.31
1     Coffee Shop  0.11
2             Art  0.08
3     Simple Food  0.06
4  Breakfast Spot  0.04


----Brockton, Exhibition Place, Parkdale Village----
         venue  freq
0   Restaurant  0.29
1          Bar  0.09
2  Simple Food  0.08
3  Coffee Shop  0.08
4          Gym  0.05


----Business Reply Mail Processing Centre 969 Eastern----
           venue  freq
0     Restaurant  0.12
1        Station  0.12
2    Yoga Studio  0.06
3  Moving Target  0.06
4     Skate Park  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
         venue  freq
0   Restaurant  0.33
1  Simple Food  0.12
2  Coffee Shop  0.09
3        Hotel  0.04
4          Bar  0.04


----Cabbagetown, St. James Town----
           venu

Looks like restaurants and simple food dominate pretty much every neighborhood.

Make a function to get the top-N venues per neighborhood:

In [162]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


# Pick "top 6" categories

Calculate similarity from the top 6 venue categories in each neighborgood.

In [163]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,"Adelaide, King, Richmond",Restaurant,Coffee Shop,Simple Food,Bar,Gym,Hotel
1,Berczy Park,Restaurant,Coffee Shop,Art,Simple Food,Breakfast Spot,Bar
2,"Brockton, Exhibition Place, Parkdale Village",Restaurant,Bar,Coffee Shop,Simple Food,Gym,Spa
3,Business Reply Mail Processing Centre 969 Eastern,Station,Restaurant,Yoga Studio,Burrito Place,Moving Target,Brewery
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Restaurant,Simple Food,Coffee Shop,Bar,Hotel,Gym


# Use k-Means to find similar neighborhoods

It seems there are only 3 or 4 different types of neighborhoods. Looking for 4:

In [164]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 0, 3, 0, 3, 3, 0, 3], dtype=int32)

# Plot results

It looks as if the neighborhoods are clustering in more than just one big blob. Plot it.

Merge neighborhoods together by their cluster label:

In [165]:
# add clustering labels

if 'Cluster Labels' in neighborhoods_venues_sorted.columns:
    neighborhoods_venues_sorted = neighborhoods_venues_sorted.drop('Cluster Labels', axis=1)

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3.0,Restaurant,Simple Food,Coffee Shop,Bar,Grocery Store,Book
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1.0,Simple Food,Restaurant,Park,Gym,Station,Pub
43,M4M,East Toronto,Studio District,43.659526,-79.340923,3.0,Restaurant,Coffee Shop,Bakery,Bar,Simple Food,Yoga Studio
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Plot using folium

Plot the result using folium (but exclude the neighborhoods that didn't have many venues from starters; they show here as cluster label "NaN"):

In [166]:
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[43.68,-79.37], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    if not np.isnan(cluster):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=10,
            popup=label,
            color=rainbow[int(cluster)-1],
            fill=True,
            fill_color=rainbow[int(cluster)-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# Image snapshot:

https://github.com/koeplinger/sandbox-ibm-ds/blob/master/c9w3_assg_pic2.png

# Inspect cluster label 0:

In [167]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
46,Central Toronto,0.0,Restaurant,Coffee Shop,Clothing Store,Simple Food,Sporting Goods Shop,Yoga Studio
49,Central Toronto,0.0,Restaurant,Grocery Store,Simple Food,Pub,Coffee Shop,Spa
51,Downtown Toronto,0.0,Restaurant,Coffee Shop,Simple Food,Park,Grocery Store,Pub
53,Downtown Toronto,0.0,Coffee Shop,Restaurant,Simple Food,Bakery,Park,Pub
75,Downtown Toronto,0.0,Grocery Store,Coffee Shop,Park,Restaurant,Baby Store,Nightlife
76,West Toronto,0.0,Pharmacy,Grocery Store,Restaurant,Bakery,Discount Store,Gym
87,East Toronto,0.0,Station,Restaurant,Yoga Studio,Burrito Place,Moving Target,Brewery


A good mix of restaurants, coffee shops, grocery, parks, and pubs.

# Inspect cluster label 1

In [168]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
42,East Toronto,1.0,Simple Food,Restaurant,Park,Gym,Station,Pub
47,Central Toronto,1.0,Simple Food,Restaurant,Coffee Shop,Pharmacy,Gourmet Shop,Indoor Play Area
65,Central Toronto,1.0,Simple Food,Coffee Shop,Restaurant,History Museum,Station,Liquor Store


Simple food dominates here.

# Inspect cluster label 2

In [169]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
84,West Toronto,2.0,Park,Station,Gym,Lake,Trail,Building


The only neighborhood with lots of greenery (park, lake, trail).

# Inspect cluster label 3

In [170]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
41,East Toronto,3.0,Restaurant,Simple Food,Coffee Shop,Bar,Grocery Store,Book
43,East Toronto,3.0,Restaurant,Coffee Shop,Bakery,Bar,Simple Food,Yoga Studio
52,Downtown Toronto,3.0,Restaurant,Bar,Coffee Shop,Simple Food,Pub,Tea
54,Downtown Toronto,3.0,Restaurant,Coffee Shop,Clothing Store,Simple Food,Bar,Gym
55,Downtown Toronto,3.0,Restaurant,Coffee Shop,Simple Food,Bar,Hotel,Clothing Store
56,Downtown Toronto,3.0,Restaurant,Coffee Shop,Art,Simple Food,Breakfast Spot,Bar
57,Downtown Toronto,3.0,Restaurant,Coffee Shop,Simple Food,Bar,Tea,Spa
58,Downtown Toronto,3.0,Restaurant,Coffee Shop,Simple Food,Bar,Gym,Hotel
59,Downtown Toronto,3.0,Restaurant,Coffee Shop,Art,Simple Food,Breakfast Spot,Bar
60,Downtown Toronto,3.0,Coffee Shop,Restaurant,Simple Food,Bar,Hotel,Pub


Restaurants and coffee shops dominate here.